In [3]:
import numpy as np
import openai
from retrying import retry
import pandas as pd
import pinecone
import itertools
from dotenv import load_dotenv

In [2]:
df = pd.read_csv("UCONN_CLUB_INFO.csv")
df.drop("Unnamed: 0", axis=1, inplace=True)
df.head()

,Club Names,Club Short Descriptions,Club Long Descriptions,Club Urls
0,4H Vernon Afterschool,Are you ready to make a difference in a child'...,4-H Vernon After School Enrichment Club throug...,https://uconntact.uconn.edu/organization/co4hv...
1,A Completely Different Note,"A Completely Different Note (CDN), UCONN's old...",NaN,https://uconntact.uconn.edu/organization/acomp...
2,A.P. Christian Youth - Avery Point Campus,We are God believing Youth who want to spread ...,Bible Studies\nSocial Gatherings\nPrayer Meeti...,https://uconntact.uconn.edu/organization/apchr...
3,Academic Achievement Center,The Academic Achievement Center assists studen...,NaN,https://uconntact.uconn.edu/organization/aac
4,Accounting Society,The Accounting Society has weekly meetings wit...,The Accounting Society works very closely with...,https://uconntact.uconn.edu/organization/accou...


In [4]:
load_dotenv()

True

In [ ]:
max_retries = 5
initial_backoff = 1  

@retry(stop_max_attempt_number=max_retries, wait_exponential_multiplier=1000, wait_exponential_max=10000)
def make_embedding_request(input_text):
    response = openai.Embedding.create(
        input=input_text,
        model="text-embedding-ada-002"
    )
    return response['data'][0]['embedding']

allclubsembds = []
shortembds = []
descembds = []
nameembds = []
for i in range(len(df)):
    cn = df.loc[i]["Club Names"]
    ct = df.loc[i]["Club Short Descriptions"]
    cd = df.loc[i]["Club Long Descriptions"]
    if isinstance(cd, str):
        try:
            cn_embedding = make_embedding_request(cn)
            nameembds.append(cn_embedding)
            ct_embedding = make_embedding_request(ct)
            shortembds.append(ct_embedding)
            cd_embedding = make_embedding_request(cd)
            descembds.append(cd_embedding)
            cnctcd = list((0.15 * np.array(cn_embedding)) + (0.35 * np.array(ct_embedding)) + (0.5 * np.array(cd_embedding)))
            allclubsembds.append(cnctcd)
        except openai.error.APIError as e:
            print("API Error occurred:", e)
    else:
        try:
            cn_embedding = make_embedding_request(cn)
            nameembds.append(cn_embedding)
            ct_embedding = make_embedding_request(ct)
            shortembds.append(ct_embedding)
            cnct = list((0.15 * np.array(cn_embedding)) + (0.85 * np.array(ct_embedding)))
            allclubsembds.append(cnct)
        except openai.error.APIError as e:
            print("API Error occurred:", e)